In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# coding: utf-8
%matplotlib inline 

import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
# pd.options.display.mpl_style = 'default'

### 读取原始数据

In [6]:
time_travel = pd.read_csv('./gy_contest_link_traveltime_training_data.txt',sep=';')
# time_travel['link_ID'] = time_travel['link_ID'].astype(str) 
time_travel.head()   

,link_ID,date,time_interval,travel_time
0,9377906285566510514,2016-05-21,"[2016-05-21 23:20:00,2016-05-21 23:22:00)",17.6
1,3377906288228510514,2016-05-21,"[2016-05-21 18:46:00,2016-05-21 18:48:00)",3.5
2,3377906284395510514,2016-05-21,"[2016-05-21 07:06:00,2016-05-21 07:08:00)",10.0
3,4377906284959500514,2016-05-21,"[2016-05-21 14:34:00,2016-05-21 14:36:00)",3.5
4,9377906282776510514,2016-05-21,"[2016-05-21 05:04:00,2016-05-21 05:06:00)",1.5


In [16]:
time_travel['interval'] = time_travel['time_interval'].apply(lambda x: x[11:20])
time_travel['hour'] = time_travel['time_interval'].apply(lambda x: int(x[11:14]))

In [17]:
time_travel.head()

,link_ID,date,time_interval,travel_time,interval,hour
0,9377906285566510514,2016-05-21,"[2016-05-21 23:20:00,2016-05-21 23:22:00)",17.6,23:20:00,23
1,3377906288228510514,2016-05-21,"[2016-05-21 18:46:00,2016-05-21 18:48:00)",3.5,18:46:00,18
2,3377906284395510514,2016-05-21,"[2016-05-21 07:06:00,2016-05-21 07:08:00)",10.0,07:06:00,7
3,4377906284959500514,2016-05-21,"[2016-05-21 14:34:00,2016-05-21 14:36:00)",3.5,14:34:00,14
4,9377906282776510514,2016-05-21,"[2016-05-21 05:04:00,2016-05-21 05:06:00)",1.5,05:04:00,5


In [12]:
time_travel.dtypes

link_ID           object
date              object
time_interval     object
travel_time      float64
interval          object
hour              object
dtype: object

### 筛选 6 7 8 点的数据

In [20]:
df = time_travel[time_travel.hour.isin([6,7,8])]
df.head()

,link_ID,date,time_interval,travel_time,interval,hour
2,3377906284395510514,2016-05-21,"[2016-05-21 07:06:00,2016-05-21 07:08:00)",10.0,07:06:00,7
39,3377906282418510514,2016-05-21,"[2016-05-21 08:56:00,2016-05-21 08:58:00)",3.5,08:56:00,8
62,4377906289813600514,2016-05-21,"[2016-05-21 08:00:00,2016-05-21 08:02:00)",2.0,08:00:00,8
70,4377906289525800514,2016-05-21,"[2016-05-21 08:10:00,2016-05-21 08:12:00)",2.2,08:10:00,8
71,4377906289525800514,2016-05-21,"[2016-05-21 08:50:00,2016-05-21 08:52:00)",3.2,08:50:00,8


In [ ]:
df.to_csv('time_interval_range_6_8.txt',index=F)

In [18]:
dates = pd.date_range("2015-01-01","2016-08-31", freq="1D")
userid = pd.DataFrame({'user_id':df['user_id'].unique()})
dates = pd.DataFrame(dates)
dates.columns = ['record_date']
userid['key'] = 0
dates['key'] = 0
full = userid.merge(dates, how='left', on = 'key')
full.drop('key',1, inplace=True)
full.head(2)

,user_id,record_date
0,1,2015-01-01
1,1,2015-01-02


In [19]:
df = pd.merge(full, df, how='left', on=['user_id','record_date'])
df['power_consumption'] = df['power_consumption'].interpolate()
df.shape

(885486, 3)

### 填充特征

In [20]:
df['dow'] = df['record_date'].apply(lambda x: x.dayofweek)
df['doy'] = df['record_date'].apply(lambda x: x.dayofyear)
df['day'] = df['record_date'].apply(lambda x: x.day)
df['month'] = df['record_date'].apply(lambda x: x.month)
df['year'] = df['record_date'].apply(lambda x: x.year)

def map_season(month):
    month_dic = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:3, 9:3, 10:4, 11:4, 12:1}
    return month_dic[month]

df['season'] = df['month'].apply(lambda x: map_season(x))

In [25]:
### 用电量聚合到一个月

base_df = df[['record_date','power_consumption']].groupby(by='record_date').agg('sum')
base_df = base_df.reset_index()
base_df.head()
base_df.shape

(609, 2)

### 拼接测试集

In [27]:
df_test = base_df[(base_df.record_date>='2016-08-01')&(base_df.record_date<='2016-08-30')]
df_test['record_date'] = pd.DataFrame(df_test['record_date']+pd.Timedelta('31 days'))
df_test.head()

,record_date,power_consumption
578,2016-09-01,4627029.0
579,2016-09-02,4618467.0
580,2016-09-03,4490739.0
581,2016-09-04,4367727.0
582,2016-09-05,4253298.0


In [28]:
base_df = pd.concat([base_df, df_test]).sort_values(['record_date'])
base_df.shape

(639, 2)

### 造特征

In [29]:
base_df['dow'] = base_df['record_date'].apply(lambda x: x.dayofweek)
base_df['doy'] = base_df['record_date'].apply(lambda x: x.dayofyear)
base_df['day'] = base_df['record_date'].apply(lambda x: x.day)
base_df['month'] = base_df['record_date'].apply(lambda x: x.month)
base_df['year'] = base_df['record_date'].apply(lambda x: x.year)

def map_season(month):
    month_dic = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:3, 9:3, 10:4, 11:4, 12:1}
    return month_dic[month]

# base_df['season'] = base_df['month'].apply(lambda x: map_season(x))

### 添加更多特征
### 按 day 计算均值


In [159]:
train = base_df

In [160]:

df_stats = base_df[['power_consumption','day']].groupby(by=['day']).agg(['mean', 'std'])
df_stats.shape

(31, 2)

In [161]:
df_stats.columns = df_stats.columns.droplevel(0)
df_stats = df_stats.reset_index()
df_stats.head()
df_stats['mean_day'] = df_stats['mean']#.shift(1)
# df_stats['2_m_mean'] = df_stats['mean'].shift(2)
df_stats['std_day'] = df_stats['std']#.shift(1)
# df_stats['2_m_std'] = df_stats['std'].shift(2)
df_stats.head()

# train = pd.merge('base_df','df_stats',on=['year','month'])
train = pd.merge(train, df_stats, how='left', on=['day'])
train.head()
train = train.drop(['mean', 'std'],axis=1)
train.head()

,record_date,power_consumption,dow,doy,day,month,year,week_period,month_period,mean_day,std_day
0,2015-01-01,2900575.0,3,1,1,1,2015,1,1,3.620987e+06,581292.542560
1,2015-01-02,3158211.0,4,2,2,1,2015,1,1,3.724284e+06,471504.202731
2,2015-01-03,3596487.0,5,3,3,1,2015,1,1,3.774477e+06,417498.917083
3,2015-01-04,3939672.0,6,4,4,1,2015,1,1,3.806080e+06,375370.780895
4,2015-01-05,4101790.0,0,5,5,1,2015,1,1,3.712768e+06,580312.573094


### 添加更多特征
### 先添加月均值和方差

In [162]:
df_stats = base_df[['power_consumption','year','month']].groupby(by=['year', 'month']).agg(['mean', 'std'])
df_stats.head()

power_consumption               
                        mean            std
year month                                 
2015 1          3.961383e+06  303629.486622
     2          2.795163e+06  769697.864999
     3          3.478852e+06  564753.747993
     4          3.768333e+06  190787.832757
     5          3.755262e+06  269088.249265

In [163]:
df_stats.columns = df_stats.columns.droplevel(0)
df_stats = df_stats.reset_index()
df_stats.head()



,year,month,mean,std
0,2015,1,3.961383e+06,303629.486622
1,2015,2,2.795163e+06,769697.864999
2,2015,3,3.478852e+06,564753.747993
3,2015,4,3.768333e+06,190787.832757
4,2015,5,3.755262e+06,269088.249265


In [164]:
df_stats['mean'] = df_stats['mean'].shift(1)
# df_stats['2_m_mean'] = df_stats['mean'].shift(2)
df_stats['std'] = df_stats['std'].shift(1)
# df_stats['2_m_std'] = df_stats['std'].shift(2)
df_stats.head()



,year,month,mean,std
0,2015,1,NaN,NaN
1,2015,2,3.961383e+06,303629.486622
2,2015,3,2.795163e+06,769697.864999
3,2015,4,3.478852e+06,564753.747993
4,2015,5,3.768333e+06,190787.832757


In [165]:
# train = pd.merge('base_df','df_stats',on=['year','month'])
train = pd.merge(train, df_stats, how='left', on=['year','month'])
train.head()

,record_date,power_consumption,dow,doy,day,month,year,week_period,month_period,mean_day,std_day,mean,std
0,2015-01-01,2900575.0,3,1,1,1,2015,1,1,3.620987e+06,581292.542560,NaN,NaN
1,2015-01-02,3158211.0,4,2,2,1,2015,1,1,3.724284e+06,471504.202731,NaN,NaN
2,2015-01-03,3596487.0,5,3,3,1,2015,1,1,3.774477e+06,417498.917083,NaN,NaN
3,2015-01-04,3939672.0,6,4,4,1,2015,1,1,3.806080e+06,375370.780895,NaN,NaN
4,2015-01-05,4101790.0,0,5,5,1,2015,1,1,3.712768e+06,580312.573094,NaN,NaN


In [ ]:
base_df = train

### 把一个月分成上中下旬3个month_periods 或者 4周总共4个week_periods
### 用它们的均值之差表示数据上下起伏趋势

In [114]:
# 先对一个月做切分
def month_period(day):
    if day>=1 and day<=10:
        return 1
    elif day>=11 and day<=20:
        return 2
    else:
        return 3

def week_period(day):
    if day>=1 and day<=7:
        return 1
    elif day>=8 and day<=14:
        return 2
    elif day>=15 and day<=21:
        return 3
    else:
        return 4

base_df['month_period'] = base_df['day'].apply(lambda x:month_period(x))
base_df['week_period'] = base_df['day'].apply(lambda x:week_period(x))

In [115]:
# base_df.head()


### 按照period求均值

In [166]:
month_period_df = base_df[['power_consumption','year','month','month_period']].groupby(by=['year', 'month', 'month_period']).agg('mean')
month_period_df = month_period_df.reset_index()
month_period_df.head()

,year,month,month_period,power_consumption
0,2015,1,1,3.825766e+06
1,2015,1,2,4.016926e+06
2,2015,1,3,4.034177e+06
3,2015,2,1,3.691394e+06
4,2015,2,2,2.338909e+06


### 求出来均值之差

In [167]:
month_period_df['month_period'] = month_period_df['power_consumption'].diff(1)
month_period_df.head()

,year,month,month_period,power_consumption
0,2015,1,NaN,3.825766e+06
1,2015,1,1.911600e+05,4.016926e+06
2,2015,1,1.725125e+04,4.034177e+06
3,2015,2,-3.427832e+05,3.691394e+06
4,2015,2,-1.352486e+06,2.338909e+06


### 填充上个月和上上个月的趋势数据

In [168]:
month_period_df = month_period_df[['year','month','month_period']].groupby(by=['year','month']).agg(lambda x: ",".join(x.apply(str).values.tolist()))
month_period_df = month_period_df.reset_index()
month_period_df[['month_period_1','month_period_2','month_period_3']] =  month_period_df['month_period'].str.split(',', expand=True) 
month_period_df = month_period_df.drop('month_period',1)

month_1_columns = [tmp+'_1month' for tmp in ['month_period_1', 'month_period_2','month_period_3']]
month_period_df[['month_period_1', 'month_period_2','month_period_3']] = month_period_df[['month_period_1', 'month_period_2','month_period_3']].shift(1)
# month_2_columns = [tmp+'_2month' for tmp in ['month_period_1', 'month_period_2','month_period_3']]
# month_period_df[month_2_columns] = month_period_df[['month_period_1', 'month_period_2','month_period_3']].shift(2)

month_period_df.head()

,year,month,month_period_1,month_period_2,month_period_3
0,2015,1,NaN,NaN,NaN
1,2015,2,nan,191160.0,17251.2545454544
2,2015,3,-342783.1545454548,-1352485.6999999997,-93716.5375000001
3,2015,4,863222.6375000002,559158.0499999998,-23522.659090909176
4,2015,5,109222.00909090927,-25469.75,96123.1000000001


In [169]:
# 合并
train =  pd.merge(train, month_period_df, how='left', on=['year','month'])
train.head()


,record_date,power_consumption,dow,doy,day,month,year,week_period,month_period,mean_day,std_day,mean,std,month_period_1,month_period_2,month_period_3
0,2015-01-01,2900575.0,3,1,1,1,2015,1,1,3.620987e+06,581292.542560,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,3158211.0,4,2,2,1,2015,1,1,3.724284e+06,471504.202731,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,3596487.0,5,3,3,1,2015,1,1,3.774477e+06,417498.917083,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,3939672.0,6,4,4,1,2015,1,1,3.806080e+06,375370.780895,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,4101790.0,0,5,5,1,2015,1,1,3.712768e+06,580312.573094,NaN,NaN,NaN,NaN,NaN


In [178]:
week_period_df = base_df[['power_consumption','year','month','week_period']].groupby(by=['year', 'month', 'week_period']).agg('mean')
week_period_df = week_period_df.reset_index()
week_period_df.head()

,year,month,week_period,power_consumption
0,2015,1,1,3.715404e+06
1,2015,1,2,4.013107e+06
2,2015,1,3,4.059844e+06
3,2015,1,4,4.028439e+06
4,2015,2,1,3.807695e+06


In [179]:
week_period_df['week_period'] = week_period_df['power_consumption'].diff(1)
week_period_df.head()

,year,month,week_period,power_consumption
0,2015,1,NaN,3.715404e+06
1,2015,1,297703.428571,4.013107e+06
2,2015,1,46736.285714,4.059844e+06
3,2015,1,-31404.271429,4.028439e+06
4,2015,2,-220744.014286,3.807695e+06


In [180]:
# 将每个月中四个周的值分为四列。
week_period_df = week_period_df[['year','month','week_period']].groupby(by=['year','month']).agg(lambda x: ",".join(x.apply(str).values.tolist()))
week_period_df = week_period_df.reset_index()
week_period_df[['week_period_1','week_period_2','week_period_3','week_period_4']] =  week_period_df['week_period'].str.split(',', expand=True) 
week_period_df = week_period_df.drop('week_period',1)

add_columns_1 = [tmp+'_1month' for tmp in ['week_period_1', 'week_period_2','week_period_3', 'week_period_4']]
week_period_df[['week_period_1','week_period_2','week_period_3','week_period_4']] = week_period_df[['week_period_1', 'week_period_2','week_period_3', 'week_period_4']].shift(1)
# add_columns_2 = [tmp+'_2month' for tmp in ['week_period_1', 'week_period_2','week_period_3', 'week_period_4']]
# week_period_df[add_columns_2] = week_period_df[['week_period_1', 'week_period_2','week_period_3', 'week_period_4']].shift(2)

week_period_df.head()



,year,month,week_period_1,week_period_2,week_period_3,week_period_4
0,2015,1,NaN,NaN,NaN,NaN
1,2015,2,nan,297703.42857142864,46736.28571428545,-31404.27142857155
2,2015,3,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
3,2015,4,645923.0714285714,821015.2142857141,-292440.07142857136,245783.85714285728
4,2015,5,-9620.857142857276,113845.42857142864,-66743.64285714272,91990.90476190485


In [181]:
# df_add_stats = pd.merge(base_df, df_stats, how='left', on=['year','month'])
# df_add_stats.head()

In [182]:
# df_add_month_period =  pd.merge(df_add_stats, month_period_df, how='left', on=['year','month'])
# df_add_month_period.head()

In [183]:
# week period 跟 month period 均加上时跑这个 
# df_final = pd.merge(df_add_stats, week_period_df, how='left', on=['year','month'])

In [184]:
# 只加 week period 时跑这个
df_final = pd.merge(train, week_period_df, how='left', on=['year','month']) 

In [185]:
df_final.head()

,record_date,power_consumption,dow,doy,day,month,year,week_period,month_period,mean_day,std_day,mean,std,month_period_1,month_period_2,month_period_3,week_period_1,week_period_2,week_period_3,week_period_4
0,2015-01-01,2900575.0,3,1,1,1,2015,1,1,3.620987e+06,581292.542560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,3158211.0,4,2,2,1,2015,1,1,3.724284e+06,471504.202731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,3596487.0,5,3,3,1,2015,1,1,3.774477e+06,417498.917083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,3939672.0,6,4,4,1,2015,1,1,3.806080e+06,375370.780895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,4101790.0,0,5,5,1,2015,1,1,3.712768e+06,580312.573094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
df_final.columns

Index(['record_date', 'power_consumption', 'dow', 'doy', 'day', 'month',
       'year', 'week_period', 'month_period', 'mean_day', 'std_day', 'mean',
       'std', 'month_period_1', 'month_period_2', 'month_period_3',
       'week_period_1', 'week_period_2', 'week_period_3', 'week_period_4'],
      dtype='object')

### 去掉不需要的数据

In [187]:
df_final = df_final[df_final.record_date>='2015-03-01']
# df_final['previous_2months_diff'] = df_final['1_m_mean']-df_final['2_m_mean']
df_final.head()

,record_date,power_consumption,dow,doy,day,month,year,week_period,month_period,mean_day,std_day,mean,std,month_period_1,month_period_2,month_period_3,week_period_1,week_period_2,week_period_3,week_period_4
59,2015-03-01,2900464.0,6,60,1,3,2015,1,1,3.620987e+06,581292.542560,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
60,2015-03-02,3334082.0,0,61,2,3,2015,1,1,3.724284e+06,471504.202731,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
61,2015-03-03,3492606.0,1,62,3,3,2015,1,1,3.774477e+06,417498.917083,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
62,2015-03-04,3597890.0,2,63,4,3,2015,1,1,3.806080e+06,375370.780895,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
63,2015-03-05,1798044.0,3,64,5,3,2015,1,1,3.712768e+06,580312.573094,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728


In [129]:
# df_final = df_final.drop(['mean','std','month_period', 'week_period'],1)
df_final = df_final.drop(['week_period','record_date'],1)
df_final.head()

,power_consumption,dow,doy,day,month,year,month_period,mean,std,month_period_1,month_period_2,month_period_3,week_period_1,week_period_2,week_period_3,week_period_4
59,2900464.0,6,60,1,3,2015,1,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
60,3334082.0,0,61,2,3,2015,1,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
61,3492606.0,1,62,3,3,2015,1,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
62,3597890.0,2,63,4,3,2015,1,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
63,1798044.0,3,64,5,3,2015,1,2.795163e+06,769697.864999,-342783.1545454548,-1352485.6999999997,-93716.5375000001,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728


In [130]:
df_final.to_csv('../train0529.csv',index=False)